In [14]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch

### Load Iris Dataset

In [15]:
from sklearn.datasets import load_iris

iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df["class"] = iris.target

iris_df.head(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [16]:
from sklearn.model_selection import train_test_split

iris_x = iris_df[iris_df.columns[:-1]].values
iris_y = iris_df[iris_df.columns[-1]].values
iris_y= pd.get_dummies(iris_y).values

x_train, x_test, y_train, y_test = train_test_split(iris_x, iris_y, test_size=0.33, shuffle=True)

x_train.shape, y_train.shape

((100, 4), (100, 3))

In [17]:
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)

y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

x_train.shape, y_train.shape

(torch.Size([100, 4]), torch.Size([100, 3]))

### Build Training Model

In [18]:
class Model(torch.nn.Module):
    def __init__(self, input=4, h1=8, h2=8, output=3):
        super().__init__()
        # Input layer is 4 (Features)
        self.layer1 = torch.nn.Linear(input, h1)
        # Hidden layer 1 is 8 neurons
        self.layer2 = torch.nn.Linear(h1,h2)
        # Hidden layer 2 is 8 neurons
        self.output = torch.nn.Linear(h2,output)

    def forward(self, x):
        x = torch.nn.functional.leaky_relu(self.layer1(x))
        x = torch.nn.functional.leaky_relu(self.layer2(x))
        x = torch.nn.functional.sigmoid(self.output(x))
        
        return x

In [19]:
model = Model()

# Set Loss Function
criterion = torch.nn.CrossEntropyLoss()

# Set Optimizeer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### Model Training

In [20]:
epochs = 100
losses = []

for i in range(epochs):
    # Predict the value
    y_pred = model.forward(x_train)

    # Added the loss value
    loss = criterion(y_pred, y_train)
    losses.append(loss.detach().numpy)

    # Back Propagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(f"Final Epoch: {i+1} and Losses: {loss}")

Final Epoch: 100 and Losses: 0.6845986843109131


In [21]:
correct = 0

with torch.no_grad():
    y_pred = model.forward(x_test)

    for i, data in enumerate(y_pred):
        if data.argmax().item() == y_test[i].argmax().item():
            correct +=1
print(f"We got {correct}/{i+1} correct!")

We got 36/50 correct!
